Source:https://github.com/ASFHyP3/hyp3-docs.git

---
# **Hyp3 InSAR preprocessing + Mintpy general workflow**
---

### **1. Project setting**

In [ ]:
from pathlib import Path
from dateutil.parser import parse as parse_date

# Set parameters
project_name = 'Ganzi'
# work_dir = Path.cwd() / project_name
work_dir = Path("/mnt/e/InSAR/Sentinel")
work_dir.mkdir(parents=True, exist_ok=True)
data_dir = work_dir / 'data'

stack_start = parse_date('2024-09-28 00:00:00Z')
stack_end = parse_date('2024-12-31 00:00:00Z')
max_temporal_baseline = 40  # days

data_dir.mkdir(parents=True, exist_ok=True)

### **2. Add ASF search**

In [ ]:
import re
def _extract_kml_polygons(file_path: str):
    """Extract polygon coordinates from KML file"""
    polygons = []

    try:
        if file_path.endswith('.kml'):
            with open(file_path) as kml:
                content = kml.read()
                if isinstance(content, bytes):
                    content = content.decode('utf-8', errors='ignore')
                coord_pattern = r'<coordinates[^>]*>(.*?)</coordinates>'
                coord_matches = re.findall(coord_pattern, content, re.DOTALL | re.IGNORECASE)

                for coord_block in coord_matches:
                    points = []
                    coord_text = coord_block.strip()
                    coord_text = re.sub(r'\s+', ' ', coord_text)
                    coord_triplets = coord_text.split()

                    for coord_str in coord_triplets:
                        if ',' in coord_str:
                            parts = coord_str.split(',')
                            if len(parts) >= 2:
                                try:
                                    lon = float(parts[0])
                                    lat = float(parts[1])
                                    if -180 <= lon <= 180 and -90 <= lat <= 90:
                                        points.append((lon, lat))
                                except ValueError:
                                    pass
                    if points:
                        # Remove duplicate closing point if exists
                        if len(points) > 1 and points[0] == points[-1]:
                            points = points[:-1]
                        polygons.append(points)

    except Exception as e:
        print(f"Error extracting KML: {e}")

    return polygons

In [ ]:
import asf_search as asf
import pandas as pd
from shapely.geometry import Polygon

# Select AOI with ASF search
polygons = _extract_kml_polygons(f'/mnt/e/InSAR/Ganzi/crop.kml')
aoi_wkt = Polygon(polygons[0]).wkt

search_results = asf.geo_search(
    platform=asf.PLATFORM.SENTINEL1,
    # intersectsWith='POINT(-117.55 35.77)',
    intersectsWith=aoi_wkt,
    start='2024-09-01',
    end='2024-12-31',
    processingLevel=asf.PRODUCT_TYPE.SLC,
    beamMode=asf.BEAMMODE.IW
)
# Baseline search
baseline_results = asf.baseline_search.stack_from_product(search_results[-1])

columns = list(baseline_results[0].properties.keys()) + ['geometry', ]
data = [list(scene.properties.values()) + [scene.geometry, ] for scene in baseline_results]

stack = pd.DataFrame(data, columns=columns)
stack['startTime'] = stack.startTime.apply(parse_date)

stack = stack.loc[(stack_start <= stack.startTime) & (stack.startTime <= stack_end)]

# Set up pairs
sbas_pairs = set()

for reference, rt in stack.loc[::-1, ['sceneName', 'temporalBaseline']].itertuples(index=False):
    secondaries = stack.loc[
        (stack.sceneName != reference)
        & (stack.temporalBaseline - rt <= max_temporal_baseline)
        & (stack.temporalBaseline - rt > 0)
    ]
    for secondary in secondaries.sceneName:
        sbas_pairs.add((reference, secondary))

print(f"Find {len(sbas_pairs)} SBAS pairs:\n")
for i in sbas_pairs:
    print(i)

### **3. Task running on Hyp3 server**
---
Notice: Before starting connection, set up `~/.netrc` as follows,
```bash
machine urs.earthdata.nasa.gov login [your_user_name] password [your_password]
```
Then run the following command to set up the permission of the file
```bash
chmod 600 ~/.netrc
```


3.1 Connect to Hyp3

In [ ]:
from hyp3_sdk import HyP3, Batch, util
hyp3 = HyP3()

3.2 Submit the job

In [ ]:
jobs = Batch()
for reference, secondary in sbas_pairs:
    jobs += hyp3.submit_insar_job(reference, secondary, name=project_name,
                                  include_dem=True, include_look_vectors=True)
print(jobs)

3.3 Watch the job

In [ ]:
jobs = hyp3.watch(jobs, timeout=8*3600, interval=60)

In [ ]:
jobs = hyp3.find_jobs(name=project_name)
print(jobs)

3.4 Download the products

In [ ]:
insar_products = jobs.download_files(data_dir)
insar_products = [util.extract_zipped_product(ii) for ii in insar_products]

### **4. Prepare for Mintpy time-series analysis**

In [ ]:
from pathlib import Path
from typing import List, Union
from osgeo import gdal


def get_common_overlap(file_list: List[Union[str, Path]]) -> List[float]:
    """Get the common overlap of  a list of GeoTIFF files
    
    Arg:
        file_list: a list of GeoTIFF files
    
    Returns:
         [ulx, uly, lrx, lry], the upper-left x, upper-left y, lower-right x, and lower-right y
         corner coordinates of the common overlap
    """
    
    corners = [gdal.Info(str(dem), format='json')['cornerCoordinates'] for dem in file_list]

    ulx = max(corner['upperLeft'][0] for corner in corners)
    uly = min(corner['upperLeft'][1] for corner in corners)
    lrx = min(corner['lowerRight'][0] for corner in corners)
    lry = max(corner['lowerRight'][1] for corner in corners)
    return [ulx, uly, lrx, lry]

In [ ]:
files = data_dir.glob('*/*_dem.tif')

overlap = get_common_overlap(files)

In [ ]:
from pathlib import Path
from typing import List, Union

def clip_hyp3_products_to_common_overlap(data_dir: Union[str, Path], overlap: List[float]) -> None:
    """Clip all GeoTIFF files to their common overlap
    
    Args:
        data_dir:
            directory containing the GeoTIFF files to clip
        overlap:
            a list of the upper-left x, upper-left y, lower-right-x, and lower-tight y
            corner coordinates of the common overlap
    Returns: None
    """

    
    files_for_mintpy = ['_water_mask.tif', '_corr.tif', '_unw_phase.tif', '_dem.tif', '_lv_theta.tif', '_lv_phi.tif']

    for extension in files_for_mintpy:

        for file in data_dir.rglob(f'*{extension}'):

            dst_file = file.parent / f'{file.stem}_clipped{file.suffix}'

            gdal.Translate(destName=str(dst_file), srcDS=str(file), projWin=overlap)

In [ ]:
clip_hyp3_products_to_common_overlap(data_dir, overlap)

In [ ]:
mintpy_config = work_dir / 'mintpy_config.txt'
mintpy_config.write_text(
f"""# vim: set filetype=cfg:
##------------------------  MintPy ----------------------------------##
########## 1. load_data
mintpy.load.processor        = hyp3
##---------interferogram datasets
mintpy.load.unwFile          = {data_dir}/*/*_unw_phase_clipped.tif
mintpy.load.corFile          = {data_dir}/*/*_corr_clipped.tif
##---------geometry datasets:
mintpy.load.demFile          = {data_dir}/*/*_dem_clipped.tif
mintpy.load.incAngleFile     = {data_dir}/*/*_lv_theta_clipped.tif
mintpy.load.azAngleFile      = {data_dir}/*/*_lv_phi_clipped.tif
mintpy.load.waterMaskFile    = {data_dir}/*/*_water_mask_clipped.tif

########## 2. modify_network
mintpy.network.coherenceBased       = yes       #[yes / no], auto for yes, exclude interferograms with coherence < minCoherence
mintpy.network.keepMinSpanTree      = yes
mintpy.network.minCoherence    = 0.4  #[0.0-1.0], auto for 0.7

########## 5. invert_network
mintpy.networkInversion.minTempCoh  = 0.5 #[0.0-1.0], auto for 0.7, min temporal coherence for mask

########## 8. correct_troposphere (optional but recommended)
mintpy.troposphericDelay.method = height_correlation  #[pyaps / height_correlation / gacos / no], auto for pyaps

########## 9. deramp (optional)
mintpy.deramp          = linear  #[no / linear / quadratic], auto for no - no ramp will be removed

########## 11.2 reference_date
## Reference all time-series to one date in time
## reference: Yunjun et al. (2019, section 4.9)
## no     - do not change the default reference date (1st date)
#mintpy.reference.date =  no #[reference_date.txt / 20090214 / no], auto for reference_date.txt
""")

### **5. Run Mintpy time-series analysis**

In [ ]:
!smallbaselineApp.py --dir {work_dir} {mintpy_config}